In [25]:
from google.colab import drive
drive.mount('/content/drive/My')


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-26-c0fc2c3766a0>", line 2, in <cell line: 2>
    drive.mount('/content/drive/MyDrive')
  File "/usr/local/lib/python3.10/dist-packages/google/colab/drive.py", line 103, in mount
    return _mount(
  File "/usr/local/lib/python3.10/dist-packages/google/colab/drive.py", line 197, in _mount
    raise ValueError('Mountpoint must be in a directory that exists')
ValueError: Mountpoint must be in a directory that exists

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'ValueError' object has no attribute '_render_traceback_'

During handling of the above exceptio

In [19]:
%cd /content/drive/MyDrive/Akash_Job_Project/_TeamFrog_submission/TeamFrog_nb_for_submission/

/content/drive/MyDrive/Akash_Job_Project/_TeamFrog_submission/TeamFrog_nb_for_submission


In [2]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import warnings
warnings.filterwarnings('ignore')

def evaluation(y_true, y_pred):
    print('\n\nMetric results:')
    print(f'MSE: {mean_squared_error(y_true, y_pred)}')
    print(f'MAE: {mean_absolute_error(y_true, y_pred)}')
    print(f'RMSE: {np.sqrt(mean_squared_error(y_true, y_pred))}')
    print(f'R2: {r2_score(y_true, y_pred)}', end='\n\n')

In [3]:
d = pd.read_csv("Home_Kitchen_title_price_including_cluster.csv")
d.head()

FileNotFoundError: ignored

# Benchmark 2: Random Forest regression

### One-hot encoding for categorical features
- We performed this step before splitting the data since the training and test datasets must have the same features.

In [ ]:
transformer = make_column_transformer(
    (OneHotEncoder(handle_unknown='ignore', sparse=False), ['brand', 'cluster']),
    remainder="passthrough"
    )
d = pd.DataFrame(transformer.fit_transform(d), columns=transformer.get_feature_names())


### Train-test data split

In [ ]:
X_tr, X_te, y_tr, y_te = train_test_split(d.loc[:, (d.columns != "price") & (d.columns != 'asin')], d.loc[:, 'price'], random_state = 0)
print(X_tr.shape)
print(X_te.shape)

(8296, 122)
(2766, 122)


### Pipeline
- Scaling numeric variables using `MinMaxScaler()`
- `RandomForestRegressor` as a regressor
- `GridSearchCV` to find out the best model

In [ ]:
scaler = ColumnTransformer(
    transformers=[('scaler', MinMaxScaler(), ['N_images', 'HighResImg', 'N_description'])],
    remainder='passthrough'
)
param_grid = {
    "regressor__n_estimators": [1000, 2000],
    "regressor__max_depth": [10, 20],
             }
p = Pipeline([
    ('scaler', scaler),
    ('regressor', RandomForestRegressor(random_state=0))
    ]
    )
gcv = GridSearchCV(p, param_grid, cv=5, n_jobs=-1)
gcv.fit(X_tr, y_tr)
print(gcv.best_params_)
print(gcv.best_score_)

{'regressor__max_depth': 10, 'regressor__n_estimators': 1000}
0.019073751364911205


### Evaluation

In [ ]:
y_pred = gcv.best_estimator_.predict(X_te)
evaluation(y_te, y_pred)



Metric results:
MSE: 6352.52770061458
MAE: 43.42384866091617
RMSE: 79.70274587876241
R2: 0.05635297601918621



NEW SECTION (NEXT BENCHMARK)



### Load packages and define functions

In [ ]:
import pandas as pd
import numpy as np

import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

import warnings
warnings.filterwarnings('ignore')

def evaluation(y_true, y_pred):
    print('\n\nMetric results:')
    print(f'MSE: {mean_squared_error(y_true, y_pred)}')
    print(f'MAE: {mean_absolute_error(y_true, y_pred)}')
    print(f'RMSE: {np.sqrt(mean_squared_error(y_true, y_pred))}')
    print(f'R2: {r2_score(y_true, y_pred)}', end='\n\n')

In [ ]:
d = pd.read_csv("Home_Kitchen_title_price_including_cluster.csv")
d.head()

,asin,brand,price,N_images,HighResImg,N_description,cluster
0,7301058462,NaN,1.41,6,1,127,83
1,7300069797,NaN,2.85,8,1,23,83
2,7806982574,NaN,6.42,11,1,23,17
3,7810296841,NaN,2.85,5,1,23,36
4,B0000225BF,NaN,6.42,0,1,183,15


### One-hot encoding for categorical features

In [ ]:
transformer = make_column_transformer(
    (OneHotEncoder(handle_unknown='ignore', sparse=False), ['brand', 'cluster']),
    remainder="passthrough"
    )
d = pd.DataFrame(transformer.fit_transform(d), columns=transformer.get_feature_names())

In [ ]:
X_tr, X_te, y_tr, y_te = train_test_split(d.loc[:, (d.columns != "price") & (d.columns != 'asin')], d.loc[:, 'price'], random_state = 0)
print(X_tr.shape)
print(X_te.shape)

X_tr = np.asarray(X_tr).astype(np.float32)
X_te = np.asarray(X_te).astype(np.float32)

(8296, 122)
(2766, 122)


### Pipeline
- Scaling numeric variables using `MinMaxScaler()`
- `RandomForestRegressor` as a regressor
- `GridSearchCV` to find out the best model

In [ ]:
xgbr = xgb.XGBRegressor(
    # n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    colsample_bytree=0.8,
    # reg_alpha=0.1,
    # reg_lambda=1,
    subsample=0.8,
    objective='reg:squarederror',
    eval_metric=mean_squared_error,
    # early_stopping_rounds=20,
    n_jobs=-1,
    random_state=0,)

In [ ]:
scaler = ColumnTransformer(
    transformers=[('scaler', MinMaxScaler(), ['N_images', 'HighResImg', 'N_description'])],
    remainder='passthrough'
)
param_grid = {
    "regressor__n_estimators": [300, 500],
    "regressor__reg_lambda": [0.1, 1, 2, 5]
             }
p = Pipeline([
    # ('scaler', scaler),
    ('regressor', xgbr)
    ]
    )
gcv = GridSearchCV(p, param_grid, cv=5, n_jobs=-1)
gcv.fit(X_tr, y_tr)
print(gcv.best_params_)

{'regressor__n_estimators': 300, 'regressor__reg_lambda': 5}


### Evaluation

In [ ]:
y_pred = gcv.best_estimator_.predict(X_te)
evaluation(y_te, y_pred)



Metric results:
MSE: 6335.9116319319555
MAE: 42.246519011561624
RMSE: 79.59843988378137
R2: 0.058821238182152014

